In [1]:
!pip install transformers -q

In [2]:
import numpy as np
import pandas as pd
import transformers
import torch
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch import nn
import warnings
warnings.filterwarnings("ignore")

In [3]:
from transformers import DistilBertModel, DistilBertTokenizer

In [4]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
bert = DistilBertModel.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

#### Loading the dataset and pre-processing

In [6]:
data = pd.read_csv('/kaggle/input/kaggle/train.csv')

In [7]:
data

,abstract,author,doi,url,publication month,publication year,title,publisher,label,data_index
0,the production of b jets in association with a...,CMS Collaboration,10.1007/JHEP06(2012)126,NaN,NaN,NaN,Measurement of the Z/gamma* + b-jet cross sect...,JHEP 06 (2012) 126,Physics,44436
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",NaN,NaN,NaN,NaN,Modelling systemic price cojumps with Hawkes f...,NaN,Quantitative Finance,46308
2,large information sizes in samples and feature...,"David Banh, Alan Huang",NaN,NaN,NaN,NaN,Encoding large information structures in linea...,NaN,Machine Learning,52468
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",NaN,NaN,NaN,NaN,Hyperbolic polygonal billiards close to 1-dime...,NaN,Dynamics/Dynamical Systems,12449
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",10.1016/j.jag.2012.08.003,https://doi.org/10.1016/j.jag.2012.08.003,4,2013.0,analysis of aster data for mapping bauxite ric...,International Journal of Applied Earth Observa...,Sociology,43248
...,...,...,...,...,...,...,...,...,...,...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",10.1002/pmj.21366,NaN,10,2013.0,recruiting project managers: a comparative ana...,NaN,Sociology,52038
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),NaN,NaN,NaN,NaN,Optimal stopping for L\'evy processes and affi...,NaN,Statistics and Probability,20925
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",10.1103/PhysRevD.78.075006,NaN,NaN,NaN,Against Tachyophobia,"Phys.Rev.D78:075006,2008",Physics,16875
41538,NaN,"['Markus Mühling ', 'Ralph Ewerth', 'Joanna Ba...",10.1007/978-3-030-30760-8_9,NaN,9,2019.0,investigating correlations of inter-coder agre...,NaN,Computer Sciences,46341


In [8]:
data.columns

Index(['abstract', 'author', 'doi', 'url', 'publication month',
       'publication year', 'title', 'publisher', 'label', 'data_index'],
      dtype='object')

In [9]:
data.isnull().sum()

abstract              1120
author                 189
doi                  16977
url                  39593
publication month    34943
publication year     34936
title                    0
publisher            23942
label                    0
data_index               0
dtype: int64

In [10]:
data.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
data

,abstract,author,title,label
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics
41538,NaN,"['Markus Mühling ', 'Ralph Ewerth', 'Joanna Ba...",investigating correlations of inter-coder agre...,Computer Sciences


In [11]:
data.isnull().sum()

abstract    1120
author       189
title          0
label          0
dtype: int64

In [12]:
data = data.dropna()
data.isnull().sum()

abstract    0
author      0
title       0
label       0
dtype: int64

In [13]:
data.shape

(40332, 4)

In [14]:
data

,abstract,author,title,label
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology
...,...,...,...,...
41534,programs offered by academic institutions in h...,"['Alex Ferworn', 'Muthana Zouri']",an ontology-based approach for curriculum mapp...,Computer Engineering
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics


In [15]:
data["text"] = data["title"] + data["abstract"]

In [16]:
data

,abstract,author,title,label,text
0,the production of b jets in association with a...,CMS Collaboration,Measurement of the Z/gamma* + b-jet cross sect...,Physics,Measurement of the Z/gamma* + b-jet cross sect...
1,instabilities in the price dynamics of a large...,"Giacomo Bormetti, Lucio Maria Calcagnile, Mich...",Modelling systemic price cojumps with Hawkes f...,Quantitative Finance,Modelling systemic price cojumps with Hawkes f...
2,large information sizes in samples and feature...,"David Banh, Alan Huang",Encoding large information structures in linea...,Machine Learning,Encoding large information structures in linea...
3,we consider polygonal billiards with collision...,"Gianluigi Del Magno, Jo\~ao Lopes Dias, Pedro ...",Hyperbolic polygonal billiards close to 1-dime...,Dynamics/Dynamical Systems,Hyperbolic polygonal billiards close to 1-dime...
4,Bauxite deposits of Jharkhand in India are res...,"['E.N. Dhanamjaya Rao', 'A.T. Jeyaseelan', 'K....",analysis of aster data for mapping bauxite ric...,Sociology,analysis of aster data for mapping bauxite ric...
...,...,...,...,...,...
41534,programs offered by academic institutions in h...,"['Alex Ferworn', 'Muthana Zouri']",an ontology-based approach for curriculum mapp...,Computer Engineering,an ontology-based approach for curriculum mapp...
41535,this research addresses the competencies organ...,"['Sabik Khan', 'Marcus Ho', 'Kamrul Ahsan']",recruiting project managers: a comparative ana...,Sociology,recruiting project managers: a comparative ana...
41536,this paper studies an optimal stopping problem...,Diana Dorobantu (LSProba),Optimal stopping for L\'evy processes and affi...,Statistics and Probability,Optimal stopping for L\'evy processes and affi...
41537,we examine the possible extension of the param...,"John Ellis, Joel Giedt, Oleg Lebedev, Keith Ol...",Against Tachyophobia,Physics,Against Tachyophobiawe examine the possible ex...


In [17]:
def NLP_cleaning(text):
    text_corpus = []
    for sent in tqdm(text, desc='Cleaning'):
        sent = re.sub('<[^>]*>', '', sent)
        sent = re.sub('[^a-zA-z0-9]', ' ', sent)
        sent = sent.lower()
        text_corpus.append(sent)

    return text_corpus

In [18]:
text = data.text.values.tolist()

In [19]:
text[0]

'Measurement of the Z/gamma* + b-jet cross section in pp collisions at 7\n  TeVthe production of b jets in association with a z/gamma* boson is studied using proton-proton collisions delivered by the lhc at a centre-of-mass energy of 7 tev and recorded by the cms detector. the inclusive cross section for z/gamma* + b-jet production is measured in a sample corresponding to an integrated luminosity of 2.2 inverse femtobarns. the z/gamma* + b-jet cross section with z/gamma* to ll (where ll = ee or mu mu) for events with the invariant mass 60 < m(ll) < 120 gev, at least one b jet at the hadron level with pt > 25 gev and abs(eta) < 2.1, and a separation between the leptons and the jets of delta r > 0.5 is found to be 5.84 +/- 0.08 (stat.) +/- 0.72 (syst.) +(0.25)/-(0.55) (theory) pb. the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia.'

In [20]:
text_corpus = NLP_cleaning(text)

Cleaning: 100%|██████████| 40332/40332 [00:03<00:00, 11884.64it/s]


In [21]:
text_corpus[0]

'measurement of the z gamma    b jet cross section in pp collisions at 7   tevthe production of b jets in association with a z gamma  boson is studied using proton proton collisions delivered by the lhc at a centre of mass energy of 7 tev and recorded by the cms detector  the inclusive cross section for z gamma    b jet production is measured in a sample corresponding to an integrated luminosity of 2 2 inverse femtobarns  the z gamma    b jet cross section with z gamma  to ll  where ll   ee or mu mu  for events with the invariant mass 60  25 gev and abs eta   0 5 is found to be 5 84     0 08  stat       0 72  syst     0 25    0 55   theory  pb  the kinematic properties of the events are also studied and found to be in agreement with the predictions made by the madgraph event generator with the parton shower and the hadronisation performed by pythia '

In [22]:
data['text'] = text_corpus

In [23]:
data['title'] = NLP_cleaning(data.title.values.tolist())
data['author'] = NLP_cleaning(data.author.values.tolist())

Cleaning: 100%|██████████| 40332/40332 [00:00<00:00, 85340.16it/s]


In [24]:
data

,abstract,author,title,label,text
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,Physics,measurement of the z gamma b jet cross sect...
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,Quantitative Finance,modelling systemic price cojumps with hawkes f...
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,Machine Learning,encoding large information structures in linea...
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems,hyperbolic polygonal billiards close to 1 dime...
4,Bauxite deposits of Jharkhand in India are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,Sociology,analysis of aster data for mapping bauxite ric...
...,...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,Computer Engineering,an ontology based approach for curriculum mapp...
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,Sociology,recruiting project managers a comparative ana...
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,Statistics and Probability,optimal stopping for l\ evy processes and affi...
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,Physics,against tachyophobiawe examine the possible ex...


In [25]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['label_number'] = label_encoder.fit_transform(data['label'])
data

,abstract,author,title,label,text,label_number
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,Physics,measurement of the z gamma b jet cross sect...,95
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,Quantitative Finance,modelling systemic price cojumps with hawkes f...,105
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,Machine Learning,encoding large information structures in linea...,67
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,Dynamics/Dynamical Systems,hyperbolic polygonal billiards close to 1 dime...,41
4,Bauxite deposits of Jharkhand in India are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,Sociology,analysis of aster data for mapping bauxite ric...,114
...,...,...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,Computer Engineering,an ontology based approach for curriculum mapp...,25
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,Sociology,recruiting project managers a comparative ana...,114
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,Statistics and Probability,optimal stopping for l\ evy processes and affi...,118
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,Physics,against tachyophobiawe examine the possible ex...,95


In [26]:
label_encoder

LabelEncoder()

In [27]:
val_df = pd.read_csv('/kaggle/input/kaggle/val.csv')
val_df

,abstract,author,doi,url,publication month,publication year,title,publisher,label,data_index
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",10.1093/mnras/stv974,NaN,NaN,NaN,Observational Properties of Simulated Galaxies...,NaN,External Galaxies,22292
1,NaN,"['Yongzhuo Huang', 'Shengwu Huang', 'Yongxin Y...",10.1007/s13233-013-1029-2,NaN,10.0,2012.0,cell-penetrating peptide-modified plga nanopar...,Macromolecular Research,Medicinal Chemistry and Pharmaceutics,4465
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,NaN,NaN,NaN,NaN,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,NaN,Quantum Physics,16166
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",NaN,NaN,NaN,NaN,Suppression of magnetic phase transition at hi...,NaN,"Atomic, Molecular and Optical Physics",19452
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,NaN,NaN,NaN,NaN,Many neighborly spheres,NaN,Discrete Mathematics and Combinatorics,43731
...,...,...,...,...,...,...,...,...,...,...
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,10.1088/1361-6471/ac4e60,NaN,NaN,NaN,Energy correlation of bottom quarks from decay...,J. Phys. G: Nucl. Part. Phys. 49 045003 (2022),Physics,49058
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,NaN,NaN,NaN,NaN,Scattering processes in antiprotonic hydrogen ...,NaN,Nonlinear Sciences,26605
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",10.14778/1453856.1453873,NaN,8.0,2008.0,resisting structural re-identification in anon...,NaN,Earth Sciences,44741
8899,Predictive models of immune protection from CO...,"['Miles P. Davenport', 'James A. Triccas', 'St...",10.1038/s41591-021-01377-8,http://dx.doi.org/10.1038/s41591-021-01377-8,5.0,2021.0,neutralizing antibody levels are highly predic...,Nature Medicine,Microbiology,19564


In [28]:
val_df.drop(['doi','url','publication month', 'publication year','publisher', 'data_index'], axis =1 , inplace = True)
val_df = val_df.dropna()


In [29]:
val_df['label_number'] = label_encoder.transform(val_df['label'])
val_df

,abstract,author,title,label,label_number
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",Observational Properties of Simulated Galaxies...,External Galaxies,51
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,Quantum Physics,107
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",Suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics",10
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,Many neighborly spheres,Discrete Mathematics and Combinatorics,40
5,"using first-principles calculations, the elast...","Ming-Min Zhong, Xiao-Yu Kuang, Zhen-Hua Wang, ...",Physical properties of ReB2 under high pressur...,Materials Science and Engineering,68
...,...,...,...,...,...
8895,we present x-ray spectral analysis of seyfert ...,"Prantik Nandi, Sandip K. Chakrabarti, Santanu ...",Spectral properties of NGC 4151 and the Estima...,Physical Processes,93
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,Energy correlation of bottom quarks from decay...,Physics,95
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,Scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,84
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",resisting structural re-identification in anon...,Earth Sciences,42


In [30]:
val_df["text"] = val_df["title"] + val_df["abstract"]

In [31]:
val_df

,abstract,author,title,label,label_number,text
0,we use high-resolution zoom-in cosmological si...,"Hidenobu Yajima (1), Isaac Shlosman (2, 3), Em...",Observational Properties of Simulated Galaxies...,External Galaxies,51,Observational Properties of Simulated Galaxies...
2,it is shown that the formulation of the su(3) ...,Hans-Peter Pavel,SU(3) Yang-Mills Hamiltonian in the flux-tube ...,Quantum Physics,107,SU(3) Yang-Mills Hamiltonian in the flux-tube ...
3,"structural, magnetization, phonon behavior, an...","Ilyas Noor Bhatti, Imtiaz Noor Bhatti, Rabindr...",Suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics",10,Suppression of magnetic phase transition at hi...
4,the result of padrol asserts that for every $d...,Isabella Novik and Hailun Zheng,Many neighborly spheres,Discrete Mathematics and Combinatorics,40,Many neighborly spheresthe result of padrol as...
5,"using first-principles calculations, the elast...","Ming-Min Zhong, Xiao-Yu Kuang, Zhen-Hua Wang, ...",Physical properties of ReB2 under high pressur...,Materials Science and Engineering,68,Physical properties of ReB2 under high pressur...
...,...,...,...,...,...,...
8895,we present x-ray spectral analysis of seyfert ...,"Prantik Nandi, Sandip K. Chakrabarti, Santanu ...",Spectral properties of NGC 4151 and the Estima...,Physical Processes,93,Spectral properties of NGC 4151 and the Estima...
8896,joint energy distribution of the bottom quark ...,Ivan V. Truten and Alexander Yu. Korchin,Energy correlation of bottom quarks from decay...,Physics,95,Energy correlation of bottom quarks from decay...
8897,"the elastic scattering, stark transitions and ...",V.P. Popov and V.N. Pomerantsev,Scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,84,Scattering processes in antiprotonic hydrogen ...
8898,We identify privacy risks associated with rele...,"['Philipp Weis', 'Don Towsley', 'David Jensen'...",resisting structural re-identification in anon...,Earth Sciences,42,resisting structural re-identification in anon...


In [32]:
val_df['title'] = NLP_cleaning(val_df.title.values.tolist())
val_df['author'] = NLP_cleaning(val_df.author.values.tolist())
val_df['abstract'] = NLP_cleaning(val_df.abstract.values.tolist())
val_df['text'] = NLP_cleaning(val_df.text.values.tolist())
val_df

Cleaning: 100%|██████████| 8648/8648 [00:00<00:00, 11865.53it/s]


,abstract,author,title,label,label_number,text
0,we use high resolution zoom in cosmological si...,hidenobu yajima 1 isaac shlosman 2 3 em...,observational properties of simulated galaxies...,External Galaxies,51,observational properties of simulated galaxies...
2,it is shown that the formulation of the su 3 ...,hans peter pavel,su 3 yang mills hamiltonian in the flux tube ...,Quantum Physics,107,su 3 yang mills hamiltonian in the flux tube ...
3,structural magnetization phonon behavior an...,ilyas noor bhatti imtiaz noor bhatti rabindr...,suppression of magnetic phase transition at hi...,"Atomic, Molecular and Optical Physics",10,suppression of magnetic phase transition at hi...
4,the result of padrol asserts that for every d...,isabella novik and hailun zheng,many neighborly spheres,Discrete Mathematics and Combinatorics,40,many neighborly spheresthe result of padrol as...
5,using first principles calculations the elast...,ming min zhong xiao yu kuang zhen hua wang ...,physical properties of reb2 under high pressur...,Materials Science and Engineering,68,physical properties of reb2 under high pressur...
...,...,...,...,...,...,...
8895,we present x ray spectral analysis of seyfert ...,prantik nandi sandip k chakrabarti santanu ...,spectral properties of ngc 4151 and the estima...,Physical Processes,93,spectral properties of ngc 4151 and the estima...
8896,joint energy distribution of the bottom quark ...,ivan v truten and alexander yu korchin,energy correlation of bottom quarks from decay...,Physics,95,energy correlation of bottom quarks from decay...
8897,the elastic scattering stark transitions and ...,v p popov and v n pomerantsev,scattering processes in antiprotonic hydrogen ...,Nonlinear Sciences,84,scattering processes in antiprotonic hydrogen ...
8898,we identify privacy risks associated with rele...,[ philipp weis don towsley david jensen ...,resisting structural re identification in anon...,Earth Sciences,42,resisting structural re identification in anon...


In [33]:
X=val_df[['text']].copy()
y=val_df[['label_number']].copy()

In [34]:
X

,text
0,observational properties of simulated galaxies...
2,su 3 yang mills hamiltonian in the flux tube ...
3,suppression of magnetic phase transition at hi...
4,many neighborly spheresthe result of padrol as...
5,physical properties of reb2 under high pressur...
...,...
8895,spectral properties of ngc 4151 and the estima...
8896,energy correlation of bottom quarks from decay...
8897,scattering processes in antiprotonic hydrogen ...
8898,resisting structural re identification in anon...


In [35]:
val_text, test_text, val_labels, test_labels = train_test_split(X,y,random_state=2018,test_size=0.4,shuffle=True)

In [36]:
test_text

,text
6069,feedback generation for performance problems i...
6807,lgvton a landmark guided approach to virtual ...
8808,an empirical analysis of exact algorithms for ...
7032,distinguishability classical information of q...
881,towards exploring literals to enrich data link...
...,...
4207,faint end quasar luminosity functions from cos...
4882,the accretion history of the universe with the...
6818,semantic assessment of smart healthcare ontolo...
2976,identified particle correlations at rhic medi...


In [37]:
train_text = data[["abstract","author","title","text"]].copy()

In [38]:
train_text

,abstract,author,title,text
0,the production of b jets in association with a...,cms collaboration,measurement of the z gamma b jet cross sect...,measurement of the z gamma b jet cross sect...
1,instabilities in the price dynamics of a large...,giacomo bormetti lucio maria calcagnile mich...,modelling systemic price cojumps with hawkes f...,modelling systemic price cojumps with hawkes f...
2,large information sizes in samples and feature...,david banh alan huang,encoding large information structures in linea...,encoding large information structures in linea...
3,we consider polygonal billiards with collision...,gianluigi del magno jo\ ao lopes dias pedro ...,hyperbolic polygonal billiards close to 1 dime...,hyperbolic polygonal billiards close to 1 dime...
4,Bauxite deposits of Jharkhand in India are res...,[ e n dhanamjaya rao a t jeyaseelan k ...,analysis of aster data for mapping bauxite ric...,analysis of aster data for mapping bauxite ric...
...,...,...,...,...
41534,programs offered by academic institutions in h...,[ alex ferworn muthana zouri ],an ontology based approach for curriculum mapp...,an ontology based approach for curriculum mapp...
41535,this research addresses the competencies organ...,[ sabik khan marcus ho kamrul ahsan ],recruiting project managers a comparative ana...,recruiting project managers a comparative ana...
41536,this paper studies an optimal stopping problem...,diana dorobantu lsproba,optimal stopping for l\ evy processes and affi...,optimal stopping for l\ evy processes and affi...
41537,we examine the possible extension of the param...,john ellis joel giedt oleg lebedev keith ol...,against tachyophobia,against tachyophobiawe examine the possible ex...


In [39]:
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]
sent_id = tokenizer.batch_encode_plus(text, padding=True)
print(sent_id)

{'input_ids': [[101, 2023, 2003, 1037, 14324, 2944, 14924, 4818, 102, 0], [101, 2057, 2097, 2986, 1011, 8694, 1037, 14324, 2944, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [40]:
val_text

,text
8140,triaxial orbit based dynamical modeling of gal...
8124,truthful equilibria in generalized common agen...
1275,unconventional magnetization of fe3o4 thin fil...
4165,learning fine grained image similarity with de...
1965,selective co2 reduction to co in water using e...
...,...
6872,neural network control techniques of hybrid ac...
3048,the rationality of the moduli space of curves ...
7529,further results on the distinctness of decimat...
951,minor and trace element emissions from post co...


In [41]:
tokens_train = tokenizer.batch_encode_plus(
    train_text['text'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

tokens_val = tokenizer.batch_encode_plus(
    val_text['text'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

tokens_test = tokenizer.batch_encode_plus(
    test_text['text'].tolist(),
    max_length = 512,
    pad_to_max_length=True,
    truncation=True
)

In [42]:
num = len(pd.unique(data['label_number']))

In [43]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(data['label_number'].tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels['label_number'].tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels['label_number'].tolist())

In [44]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [45]:
import torch

def print_model_layers(model):
    for name, param in model.named_parameters():
        print(f"Layer Name: {name}, Size: {param.size()}")

print_model_layers(bert)

Layer Name: embeddings.word_embeddings.weight, Size: torch.Size([30522, 768])
Layer Name: embeddings.position_embeddings.weight, Size: torch.Size([512, 768])
Layer Name: embeddings.LayerNorm.weight, Size: torch.Size([768])
Layer Name: embeddings.LayerNorm.bias, Size: torch.Size([768])
Layer Name: transformer.layer.0.attention.q_lin.weight, Size: torch.Size([768, 768])
Layer Name: transformer.layer.0.attention.q_lin.bias, Size: torch.Size([768])
Layer Name: transformer.layer.0.attention.k_lin.weight, Size: torch.Size([768, 768])
Layer Name: transformer.layer.0.attention.k_lin.bias, Size: torch.Size([768])
Layer Name: transformer.layer.0.attention.v_lin.weight, Size: torch.Size([768, 768])
Layer Name: transformer.layer.0.attention.v_lin.bias, Size: torch.Size([768])
Layer Name: transformer.layer.0.attention.out_lin.weight, Size: torch.Size([768, 768])
Layer Name: transformer.layer.0.attention.out_lin.bias, Size: torch.Size([768])
Layer Name: transformer.layer.0.sa_layer_norm.weight, Size

In [46]:
for name, param in bert.named_parameters():
    print(f"{name}: {param.requires_grad}")

embeddings.word_embeddings.weight: True
embeddings.position_embeddings.weight: True
embeddings.LayerNorm.weight: True
embeddings.LayerNorm.bias: True
transformer.layer.0.attention.q_lin.weight: True
transformer.layer.0.attention.q_lin.bias: True
transformer.layer.0.attention.k_lin.weight: True
transformer.layer.0.attention.k_lin.bias: True
transformer.layer.0.attention.v_lin.weight: True
transformer.layer.0.attention.v_lin.bias: True
transformer.layer.0.attention.out_lin.weight: True
transformer.layer.0.attention.out_lin.bias: True
transformer.layer.0.sa_layer_norm.weight: True
transformer.layer.0.sa_layer_norm.bias: True
transformer.layer.0.ffn.lin1.weight: True
transformer.layer.0.ffn.lin1.bias: True
transformer.layer.0.ffn.lin2.weight: True
transformer.layer.0.ffn.lin2.bias: True
transformer.layer.0.output_layer_norm.weight: True
transformer.layer.0.output_layer_norm.bias: True
transformer.layer.1.attention.q_lin.weight: True
transformer.layer.1.attention.q_lin.bias: True
transforme

In [47]:
import torch
import torch.nn as nn
from transformers import BertModel

class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 123)  # Adjust the number of output neurons for 123 classes
        self.log_softmax = nn.LogSoftmax(dim=1)  # Use dim=1 for LogSoftmax in classification

    def forward(self, sent_id, mask):
        sent_id = torch.tensor(sent_id)

        outputs = self.bert(input_ids=sent_id, attention_mask=mask)
        last_hidden_state_cls = outputs[0][:, 0, :]

        x = self.fc1(last_hidden_state_cls)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.log_softmax(x)

        return x

In [48]:
model = BERT_Arch(bert)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [49]:
device

'cuda'

In [50]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),
                  lr = 1e-4)

In [51]:
train_labels = data['label_number']

In [52]:
model

BERT_Arch(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in_fea

In [53]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(train_labels)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)
print("Class Weights:", class_weights)

Class Weights: [6.78887037e-01 5.90815205e-01 5.46504065e-01 5.12347561e+00
 5.34043060e-01 2.42890696e+00 7.14384399e-01 6.30581614e+00
 1.25922596e-01 8.62901155e+00 3.48091761e-01 2.73252033e+01
 3.71771473e-01 2.02408913e+00 5.46504065e+01 8.40775485e+00
 4.68432056e+01 1.68155097e+00 3.27902439e+01 1.36626016e+01
 1.82168022e+00 7.45232816e+00 5.28874902e+00 4.52903921e-01
 5.85540070e+00 2.29302405e+00 3.81281906e+00 7.66127194e-01
 2.80258495e-01 2.35901035e+00 2.50307205e+00 3.73891037e-01
 3.99881023e+00 5.56710423e-01 3.95539733e-01 8.40775485e-01
 1.36626016e+01 1.38941711e+00 6.31796607e-01 6.07226739e+00
 4.15592445e-01 2.70323528e-01 4.25847323e+00 1.59952409e+00
 1.26116323e+01 1.63951220e+01 2.46543187e+00 9.10840108e+00
 1.63951220e+02 1.85255615e+00 4.09878049e+00 4.23099921e-01
 1.18805232e+00 1.21445348e+01 3.51449560e-01 5.85540070e+00
 1.21445348e+01 8.40775485e+00 1.21445348e+01 2.39344846e+00
 8.86222808e+00 1.05434868e+00 3.76899355e+00 9.39548536e-01
 2.802584

In [54]:
weights= torch.tensor(class_weights,dtype=torch.float)
weights = weights.to(device)
cross_entropy  = nn.NLLLoss(weight=weights)

epochs = 5

In [55]:
def train():

    model.train()
    total_loss, total_accuracy = 0, 0
    total_preds=[]

    for step,batch in enumerate(train_dataloader):

        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        batch = [r.to(device) for r in batch]

        sent_id, mask, labels = batch
        sent_id = torch.tensor(sent_id)

        model.zero_grad()
        preds = model(sent_id, mask)
        loss = cross_entropy(preds, labels)
        total_loss = total_loss + loss.item()
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        preds=preds.detach().cpu().numpy()
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [56]:
import time
def evaluate():

    print("\nEvaluating...")
    model.eval()
    total_loss, total_accuracy = 0, 0
    total_preds = []

    for step,batch in enumerate(val_dataloader):

        if step % 50 == 0 and not step == 0:
#             elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch
    sent_id = torch.tensor(sent_id)

    with torch.no_grad():

        preds = model(sent_id, mask)
        loss = cross_entropy(preds,labels)
        total_loss = total_loss + loss.item()
        preds = preds.detach().cpu().numpy()
        total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)

    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [57]:
best_valid_loss = float('inf')

train_losses=[]
valid_losses=[]

for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, _ = train()
    valid_loss, _ = evaluate()
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 5
  Batch    50  of  2,521.
  Batch   100  of  2,521.
  Batch   150  of  2,521.
  Batch   200  of  2,521.
  Batch   250  of  2,521.
  Batch   300  of  2,521.
  Batch   350  of  2,521.
  Batch   400  of  2,521.
  Batch   450  of  2,521.
  Batch   500  of  2,521.
  Batch   550  of  2,521.
  Batch   600  of  2,521.
  Batch   650  of  2,521.
  Batch   700  of  2,521.
  Batch   750  of  2,521.
  Batch   800  of  2,521.
  Batch   850  of  2,521.
  Batch   900  of  2,521.
  Batch   950  of  2,521.
  Batch 1,000  of  2,521.
  Batch 1,050  of  2,521.
  Batch 1,100  of  2,521.
  Batch 1,150  of  2,521.
  Batch 1,200  of  2,521.
  Batch 1,250  of  2,521.
  Batch 1,300  of  2,521.
  Batch 1,350  of  2,521.
  Batch 1,400  of  2,521.
  Batch 1,450  of  2,521.
  Batch 1,500  of  2,521.
  Batch 1,550  of  2,521.
  Batch 1,600  of  2,521.
  Batch 1,650  of  2,521.
  Batch 1,700  of  2,521.
  Batch 1,750  of  2,521.
  Batch 1,800  of  2,521.
  Batch 1,850  of  2,521.
  Batch 1,900  of  2,521

In [58]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'distilbert_full_uf.pth')

In [59]:
input_text = input('Give an input text : ')
tokens = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

Give an input text :  ss


In [68]:
def Predict(text):
    encoded_review = tokenizer.encode_plus(
      text,
      max_length=512,
      add_special_tokens=True,
      return_token_type_ids=False,
      truncation = True,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, prediction = torch.max(output, dim=1)
    extracted_value = prediction.item()
    return extracted_value

In [69]:
y_pred = []
for text in test_text['text']:
    val = Predict(text)
    y_pred.append(val)

In [70]:
len(y_pred)

3460

In [71]:
test_labels

,label_number
6069,101
6807,28
8808,120
7032,107
881,6
...,...
4207,33
4882,8
6818,21
2976,84


In [72]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef, classification_report
import numpy as np

true_labels = test_labels
predicted_labels = y_pred

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5925


In [73]:
print(classification_report(true_labels, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.67      0.68        43
           1       0.91      0.64      0.75        50
           2       0.40      0.72      0.52        53
           3       1.00      1.00      1.00         7
           4       0.22      0.41      0.29        44
           5       0.25      0.56      0.34         9
           6       0.31      0.29      0.30        34
           7       0.60      0.50      0.55         6
           8       0.59      0.22      0.32       220
           9       1.00      0.50      0.67         4
          10       0.44      0.73      0.55        95
          12       0.91      0.65      0.76        78
          13       0.50      0.33      0.40        12
          14       0.00      0.00      0.00         1
          15       0.25      0.50      0.33         4
          16       0.00      0.00      0.00         1
          17       0.60      0.38      0.46        16
          19       0.00    